## Graphs and sessions

TF 1.14 的图和会话还是要熟悉，不然历史遗留代码看不懂呀。

TensorFlow 使用**数据流图**将计算表示为独立的指令之间的依赖关系。在这种编程模型中，你首先需要先定义数据流图，然后创建 TensorFlow 会话，然后就可以本地和远程设备上运行图中的各个部分了。

### 为什么使用数据流图？

<img src="https://www.tensorflow.org/images/tensors_flowing.gif">

[数据流(Dataflow Programming)](https://en.wikipedia.org/wiki/Dataflow_programming) 是一种用于并行计算的常用编程模型。在数据流图中：

- 节点，表示计算单元
- 边，表示计算使用或产生的数据

如：在 TF 图中， `tf.matmul` 操作对应于单个节点，该节点有两个输入边（要相乘的矩阵）和一个输出边（乘法结果）

在执行程序时，数据流图可以为 TF 提供很多优势：

- **并行处理**，通过使用明确的边来表示操作之间的依赖关系，系统可以轻松识别可以并行执行的操作。
- **分布式执行**, 通过使用明确的边来表示操作之间流动的值， TF 可以将程序划分到连接至不同机器的多台设备上 （CPU, GPU ，TPU)。TF 将在这些设备之间上进行必要的通信和协调。
- **编译优化**，ensorFlow 的 XLA 编译器可以使用数据流图中的信息生成更快的代码，例如将相邻的操作融合到一起。
- **可移植性**, 数据流图是一种不依赖于语言的模型代码表示法。您可以使用 Python 构建数据流图，将其存储在 SavedModel 中，并使用 C++ 程序进行恢复，从而实现低延迟的推理。

### 构建 `tf.Graph`

大多数 TF 程序都以数据流图构建阶段开始。通过调用 TF API 函数，将计算节点 (tf.Operation) 和 边(tf.Tensor) 添加到 `.tf.Graph` 中，TF 提供了一个**默认图**。

如：

- 调用 `tf.constant(42)` 创建单个 `tf.Operation`，该操作可以生成值 42.0，将该值添加到默认图中，并返回表示常量值的 tf.Tensor。

- 调用 `tf.matmul(x, y)` 可创建单个 `tf.Operation`，该操作会将 tf.Tensor 对象 x 和 y 的值相乘，将其添加到默认图中，并返回表示乘法运算结果的 tf.Tensor。

- 执行 `v = tf.Variable(0)` 可以向图添加一个 `tf.Operation`, 该操作可以存储一个可写入的张量值，该值在多个 `tf.Session.run` 调用之间保持恒定。

- 调用 `tf.train.Optimizer.minimize()` 可以将操作和张量添加到计算梯度的默认图中，并返回一个 `tf.Operation` 节点，该操作在运行时会将这些梯度应用到一组变量上。


### 命名指令

`tf.Graph` 对象会定义一个命名空间。TF 自动为图中每个指令选择一个唯一名称作为标识，使您的程序阅读和调试起来更加轻松。TF API 提供两种方法来定义操作名称：

- 如果 API 函数是创建新的 `tf.Operation` 或返回新的 `tf.Tensor` ，则可以通过 name 参数来命名。如果出现重复，TF 会自动在后面添加上数字序号。

- 通过 `tf.name_scope` 可以添加名称作用域前缀。

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

下面的代码，用于说明命名的规则。**注意**，因为 TF 为我们提供一个默认图，所有的节点和边都会添加这个图中，所以执行多次，会添加多次。

In [3]:
# 自定义名称
c1 = tf.constant(0, name="c")
c2 = tf.constant(0, name="c")
print(c1)
print(c2)

with tf.name_scope("outer"):
    c3 = tf.constant(2, name="c")
    with tf.name_scope("inner"):
        c4 = tf.constant(2, name="c")
print(c3)
print(c4)

Tensor("c:0", shape=(), dtype=int32)
Tensor("c_1:0", shape=(), dtype=int32)
Tensor("outer/c:0", shape=(), dtype=int32)
Tensor("outer/inner/c:0", shape=(), dtype=int32)


通过对指令进行分组后，通过 TensorBoard 可以更好的调试分析。

In [4]:
x = tf.constant([[37.0, -23.0], [1.0, 4.0]]) # node
w = tf.Variable(tf.random_uniform([2, 2])) # node
y = tf.matmul(x, w) # ndoe
output = tf.nn.softmax(y) # node
init_op = w.initializer # node

with tf.Session() as sess:
    # Run fetch node
    sess.run(init_op)
    print(sess.run(output))

[[1.0000000e+00 9.0960656e-11]
 [1.3551073e-01 8.6448926e-01]]


### 将操作放置到不同的设备上

我们还可以使用 `tf.device()` 函数，将创建的操作放到指定的设备上执行。设备规范如下：
 
``` 
/job:<JOB_NAME>/task:<TASK_INDEX>/device:<DEVICE_TYPE>:<DEVICE_INDEX>
```

如指定 CPU, GPU 等。电脑不给力，没办法举例。


### 使用多个图进行编程

TensorFlow 提供了一个“默认图”，对于许多应用而言，单个图已足够。但是一些高级应用，或者我们演示可能需要创建多个图。

可以调用 `tf.Graph()` 来创建一个图。


### 展示我们的计算图

#### 例子 a + b

In [10]:
g1 = tf.Graph()
with g1.as_default():
    a = tf.constant(3.1, name="a")
    b = tf.placeholder(tf.float32, name="b")
    # 使用函数而不是重载的 `+` 是因为我们可以指定 `name`
    s = tf.add(a, b, name="sum") 
    
    with tf.Session() as sess:
        writer = tf.summary.FileWriter("./logs/sum", sess.graph)
        print(sess.run(s, feed_dict={b: 4}))


7.1


我们的可视图都生成在 `logs` 目录下，可以通完 `tensorboard --logdir ./logs` 来查看。

<img src="./sum.png" width="320px" />

In [13]:
g2 = tf.Graph()
with g2.as_default():   
    x = tf.constant([[37.0, -23.0], [1.0, 4.0]], name="x") # node
    w = tf.Variable(tf.random_uniform([2, 2]), name="w") # node
    y = tf.matmul(x, w, name="inner_product") # ndoe
    output = tf.nn.softmax(y) # node
    init_op = w.initializer # node

    with tf.Session() as sess:
        writer = tf.summary.FileWriter("./logs/ex1", sess.graph)
        # Run fetch node
        sess.run(init_op)
        print(sess.run(output))

[[9.99999285e-01 6.71411215e-07]
 [8.91366303e-01 1.08633675e-01]]


然后在控制到中通过 `tensorboard --logdir ./log/ex1` 就可查看我们的计算图了。

In [14]:
g3 = tf.Graph()
with g3.as_default():    
    a = tf.placeholder(tf.float32)
    b = tf.placeholder(tf.float32)
    z = a + b
    
    with tf.Session() as sess:
        writer = tf.summary.FileWriter("./logs/ex3", sess.graph)
        print(sess.run(z, feed_dict={a: 3.1, b: 2.2}))

5.3
